# 予測

In [1]:
import cv2
import time
import argparse
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter
import chainer
from chainer import cuda, serializers, functions as F
import chainer.links as L
from chainer import Chain, Variable, cuda, optimizer, optimizers, serializers
import pickle
import os
from chainer.links import caffe

def load_list(path):
    tuples = []
    for line in open(path):
        pair = line.replace("\n","")#.strip().split()
        #print(pair)
        tuples.append(pair)
    return tuples

#print(load_list("train.txt"))

def pos_feat_diff(feat,b_po,af_po):#Position feature difference
    batch,ch,w,h = feat.data.shape
    b_po = F.broadcast_to(b_po,(batch,ch,w,h)) #(1,1,46,46) to (1,128,46,46)
    af_po = F.broadcast_to(af_po,(batch,ch,w,h))
    feat = feat - b_po + af_po
    return feat

def pos_feat_sum(po):#Position Features Summary  shape (19,320,320)
    po = F.sum(po,axis=0,keepdims=True)
    po = F.resize_images(F.expand_dims(po,axis=0),(46,46))
    return po #shape (1,1,46,46)

class MEVIN(chainer.Chain):
    insize = 320

    def __init__(self):
        super(MEVIN, self).__init__(
            mlp1_1 = chainer.ChainList(
                *[L.Linear(None,46*46)
                  for i in range(19)]),
            mlp1_2 = chainer.ChainList(
                *[L.Linear(None,4)
                  for i in range(19)]),
            mlp2_1 = chainer.ChainList(
                *[L.Linear(None,4)
                  for i in range(19*19)]),
            mlp2_2 = chainer.ChainList(
                *[L.Linear(None,2)
                  for i in range(19*19)]),
            mlp3_1 = L.Linear(None,46*46),
            mlp3_2 = L.Linear(None,46*46),
            conv1 = L.Convolution2D(in_channels=1, out_channels=19, ksize=3, stride=1, pad=1),
            conv1_a = L.Convolution2D(in_channels=19, out_channels=19, ksize=1, stride=1),
            conv1_b = L.Convolution2D(in_channels=19, out_channels=19, ksize=1, stride=1),
            conv2 = L.Convolution2D(in_channels=19, out_channels=19, ksize=3, stride=1, pad=1),
            conv2_a = L.Convolution2D(in_channels=19, out_channels=19, ksize=1, stride=1),
            conv2_b = L.Convolution2D(in_channels=19, out_channels=19, ksize=1, stride=1),
            conv3 = L.Convolution2D(in_channels=19, out_channels=19, ksize=3, stride=1, pad=1),
            conv3_a = L.Convolution2D(in_channels=19, out_channels=19, ksize=1, stride=1),
            conv3_b = L.Convolution2D(in_channels=19, out_channels=19, ksize=1, stride=1),
        )

    def __call__(self, x1,x2,x3,x4):#Pred = [x1,x2,x3,x4]
        #print(x1.shape)
        h1 = self.Mix_pofe(x1,x2,x3)
        h2 = self.Mix_pofe(x2,x3,x4)
        
        #print(len(h1))
        h1 = self.Interaction(h1)
        h2 = self.Interaction(h2)
        
        h = self.Aggregator(h1,h2) # (1,1,42,42)
        
        return h
    
    
    def Mix_pofe(self,b_x,af_x,aff_x):        
        b_x_list = F.split_axis(b_x, 19, axis=0) #[(1,320,320),......]
        af_x_list = F.split_axis(af_x, 19, axis=0) #[(1,320,320),......]
        aff_x_list = F.split_axis(aff_x, 19, axis=0)
        
        afb_x = []
        for i in range(len(b_x_list)):
            aff_af = F.concat((af_x_list[i],aff_x_list[i]),axis=1)
            afb_x.append(F.expand_dims(F.concat((b_x_list[i],aff_af),axis=1),axis=0)) #(1,3,320,320)
            
        return afb_x
    
    def Interaction(self,afb_x): 
        F_mlp = []
        for i in range(len(afb_x)):
            img = F.resize_images(afb_x[i],(46,46))
            pred = F.leaky_relu(self.mlp1_1[i](img))#(1,3,46,46) >> (1,46*46)
            pred = F.leaky_relu(self.mlp1_2[i](pred))#(1,24*24) >> (1,4)
            F_mlp.append(pred)
        
        self_inter = []
        inter = 0
        for i in range(len(F_mlp)):
            for ii in range(len(F_mlp)):
                #print(i)
                #print(ii)
                if i == ii:
                    h = F.concat((F_mlp[i],F_mlp[ii]),axis=1)
                    h = F.leaky_relu(self.mlp2_1[i * 19 + ii](h)) #(1,4)
                    #print("chack",h)
                    self_inter.append(self.mlp2_2[i * 19 + ii](h)) #(1,2)
                elif i != ii:
                    h = F.concat((F_mlp[i],F_mlp[ii]),axis=1)
                    h = F.leaky_relu(self.mlp2_1[i * 19 + ii](h)) #(1,4)
                    #print("test",h)
                    inter += F.leaky_relu(self.mlp2_2[i * 19 + ii](h))#(1,2)
                                          
        for iii in range(len(self_inter)):
            self_inter[iii] += inter
                                          
        self_inter_add_inter = F.stack(self_inter, axis=1)
        
        return self_inter_add_inter #(1,19*2)
                    
    def Aggregator(self,si_add1,si_add2):
        h = F.concat((si_add1,si_add2),axis=1)#(19,2*2)
        h = F.leaky_relu(self.mlp3_1(h)) #(1,46*46)
        h = F.leaky_relu(self.mlp3_2(h)) #(1,46*46)
        h = F.reshape(h,(1,1,46,46))
        
        h2 = F.leaky_relu(self.conv1(h))
        h2 = F.leaky_relu(self.conv1_a(h2))    
        h2 = F.leaky_relu(self.conv1_b(h2))
        h3 = F.leaky_relu(self.conv2(h2))
        h3 = F.leaky_relu(self.conv2_a(h3))    
        h3 = F.leaky_relu(self.conv2_b(h3))    
        h4 = F.leaky_relu(self.conv2(h3 + h2))
        h4 = F.leaky_relu(self.conv2_a(h4))    
        h4 = F.leaky_relu(self.conv2_b(h4)) 
        
        return h,h4
print("OK")

OK


In [2]:
model = MEVIN()
serializers.load_npz("mevin/model600.npz", model)

# GPUのセット
FLAG_GPU = False # GPUを使用するかどうか
if FLAG_GPU: # numpyかcuda.cupyか
    xp = cuda.cupy
    cuda.get_device(0).use()
    model.to_gpu()
else:
    xp = np

print("OK")

OK


## 予測スタートフレーム設定

In [19]:
#スタートフレーム
start_f = 30

#予測フレーム数
n_frame = 4

print("OK")

OK


In [20]:
batch = 1

train_list = load_list("train.txt")

Preds = []
for i in range(4): 
    ii = i + start_f
    with open('use_data_analis/'+ train_list[ii] +'_po.pickle', mode='rb') as f:
        print(train_list[ii])
        po = pickle.load(f)
        if FLAG_GPU:
            po = cuda.to_gpu(po)
    Preds.append(po)

for i in range(n_frame):
    print(i)
    with chainer.using_config('train', 'False'):
        result, pre_po = model(Preds[0],Preds[1],Preds[2],Preds[3])

    Preds.pop(0)
    Preds.append(gaussian_filter(F.resize_images(pre_po,(320,320)).data[0], sigma=2.5))
            
    if FLAG_GPU:
        reimg = cuda.to_cpu(F.resize_images(pos_feat_sum(pre_po[0]),(210,210)).data)

    else:
        reimg = F.resize_images(pos_feat_sum(pre_po[0]),(210,210)).data
                    
    cv2.imwrite('result_fillter/reimg'+str(i)+'.png', (reimg.transpose(0,2,3,1)[0])*256)
                    

1_42157.jpg
1_42158.jpg
1_42159.jpg
1_42160.jpg
0
1
2
3
